In [1]:
#import statements
import csv, random
from copy import deepcopy

In [81]:
#termination criteria is when fitness < 0.01
COUNT = [10]
def randomOp():
        options = ['+', '-', '*', '/', 'x', '-2', '-1', '1', '2']
        length = len(options)
        #generate random operation
        value = random.randint(0,length-1)
        return options[value]

class Tree:
    #initialize
    print("initialize tree")
    # make sure tree class is immutable
    def __init__(self):
        #initial tree -> call random tree generator
        #mutated tree -> call clone; makes copy of tree with changes
        # Node root
        self.root = Node()
        self.root.value = randomOp()
        self.array = []
        #self.depth = 0
        
        self._initialTree(self.root)
        

    def evaluate(x):
        #evaluates fitness of a tree
        #needs to be very efficient

        #scan/traverse tree for operators and look at left and right (its children)
        #incorporate order of operations
        #use if statements to determine next actions
        #recursive function to solve subtrees
        #ex: if + then add recursice simplification of one subtree to the simplification of other subtree
        
        #returns float or int
        val = 0
        return val

    def firstVal(self):
        start = Node()
        #set root of tree to first val
        self.root = start

        #generate random operation
        val = randomOp()
        #set node value
        start.value = val
        print(start.value)
        return start

    def _initialTree(self, node):
        operators = ['+', '-', '*', '/']
        operands = ['x', '-2', '-1', '1', '2']
        
        if (node.value in operators):
            #print("enter operator = ", node.value)
            
            leftNode = Node()
            leftNode.value = randomOp()
            node.left = leftNode
            leftNode.parent = node
            
            rightNode = Node()
            rightNode.value = randomOp()
            node.right = rightNode
            rightNode.parent = node
            
            self._initialTree(leftNode)
            self._initialTree(rightNode)

    def initialPop(self):
        #calls initial tree to create initial population
        forrest = []
        for i in range(10):
            newTree = Tree()
            forrest.append(newTree)
        #print(forrest[0].root.value)

    def printIO(self, node):
        #prints in order traversal of the tree
        if node:
            self.printIO(node.left)

            print(node.value)

            self.printIO(node.right)
    
    def treeToArray(self, root):
        #adds elements of tree to array
        if root is None:
            return
        if root:
            self.treeToArray(root.left)

            self.array.append(root)

            self.treeToArray(root.right)
    
    def randomNode(self):
        #picks random node from the tree
        length = len(self.array)
        #print("tree array: ", tree.array)
        #print("length: ", length)
        if length == 1:
            return self.array[0]
        value = random.randint(0,length-1)
        return self.array[value]

    def IOtraversal():
        # IN ORDER TREE TRAVERSAL
        # looks at node the second time you hit it
        # build string as you traverse; when you hit leaf node, add to string
        # when you hit parent for second time, add to string
        return
    
    def crossover(self, tree2):
        #takes two trees and swaps parent tree 1 with parent tree 2

        #makes copy of tree to crossover
        dup1 = deepcopy(self)
        dup1.treeToArray(dup1.root)
        dup2 = deepcopy(tree2)
        dup2.treeToArray(dup2.root)

        #find random node from tree
        random1 = dup1.randomNode()
        print("random1 node: ", random1.value)
        random2 = dup2.randomNode()
        print("random2 node: ", random2.value)

        #check parents of random nodes and change pointers to right or left
        if random1.parent == None:
            dup1.root = random2
        elif random1.parent.left == random1:
            random1.parent.left = random2
        else:
            random1.parent.right = random2
        
        if random2.parent == None:
            dup2.root = random1
        elif random2.parent.left == random2:
            random2.parent.left = random1
        else:
            random2.parent.right = random1

        #switch pointers to parents between crossover nodes
        print("original random 1 parent", random1.parent)
        print("original random2 parent", random2.parent)

        tempParent = random1.parent
        random1.parent = random2.parent
        print("Mod random 1 parent", random2.parent)
        random2.parent = tempParent
        print("Mod random 2 parent", random2.parent)

        return dup1, dup2
    
    def print2DUtil(self, root, space):
        # Base case
        if (root == None):
            return
        # Increase distance between levels
        space += COUNT[0]
    
        # Process right child first
        self.print2DUtil(root.right, space)
    
        # Print current node after space
        # count
        print()
        for i in range(COUNT[0], space):
            print(end=" ")
        print(root.value)
    
        # Process left child
        self.print2DUtil(root.left, space)

    def print2D(self, root):
        # space=[0]
        # Pass initial space count as 0
        self.print2DUtil(root, 0) 

class Node:
    #initialize
    print("initialize node")
    def __init__(self):
        # Node parent, left, right
        self.parent = None
        self.left = None
        self.right = None
        # Value: contents of the node (type: str)
        self.value = None 
    

initialize tree
initialize node


In [83]:
def main():
    with open('dataset1.csv', newline='') as csvfile:
        data = list(csv.reader(csvfile))
    data = data[1:]
    train = data[0:20000]
    test = data[20000:]
    #print(data[0:10])
    print("FIRST TREE")
    tree1 = Tree()
    #tree1.initialPop()
    tree1.print2D(tree1.root)
    #treeToArray(tree1, tree1.root)
    #print("tree1 array", tree1.array[0].value)
    print("SECOND TREE")
    tree2 = Tree()
    #tree2.initialPop()
    tree2.print2D(tree2.root)

    arr = tree1.crossover(tree2)

    print("MOD TREE 1")
    arr[0].print2D(arr[0].root)
    print("MOD TREE 2")
    arr[1].print2D(arr[1].root)


    #printIO(tree.root)
    
    #treeToArray(tree1, tree1.root)
    #print(tree1.array)

if __name__ == "__main__":
    main()

FIRST TREE

          1

*

                    -1

          +

                              -2

                    -

                              1
SECOND TREE

          2

*

          1
random1 node:  -
random2 node:  *
original random 1 parent <__main__.Node object at 0x7fd46a185df0>
original random2 parent None
Mod random 1 parent None
Mod random 2 parent <__main__.Node object at 0x7fd46a185df0>
MOD TREE 1

          1

*

                    -1

          +

                              2

                    *

                              1
MOD TREE 2

          -2

-

          1
